## Evaluate the metrics on datasets

In [ ]:
import torch


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# set up configs and weights
model_cfgs =  {
    "CARLA": {
        "model": "../../config/segmentation/carla/unet_carla_benign.py",
        "weights": "../../scripts/segmentation/models/carla/unet_carla_benign",
    },
    "nuScenes": {
        "model": "../../config/segmentation/nuscenes/unet_nuscenes_benign.py",
        "weights": "../../scripts/segmentation/models/nuscenes/unet_nuscenes_benign",
    },
    "UGV": {
        "model": "../../config/segmentation/ugv/unet_ugv_benign.py",
        "weights": "../../scripts/segmentation/models/ugv/unet_ugv_benign",
    },
    "CARLA_mc": {
        "model": "../../config/segmentation/carla/unet_carla_benign_mc.py",
        "weights": "../../scripts/segmentation/models/carla/unet_carla_benign_mc",
    },
    "nuScenes_mc": {
        "model": "../../config/segmentation/nuscenes/unet_nuscenes_benign_mc.py",
        "weights": "../../scripts/segmentation/models/nuscenes/unet_nuscenes_benign_mc",
    },
    "UGV_mc": {
        "model": "../../config/segmentation/ugv/unet_ugv_benign_mc.py",
        "weights": "../../scripts/segmentation/models/ugv/unet_ugv_benign_mc",
    },
    "CARLA_adv": {
        "model": "../../config/segmentation/carla/unet_carla_adversarial.py",
        "weights": "../../scripts/segmentation/models/carla/unet_carla_adversarial",
    },
    "nuScenes_adv": {
        "model":"../../config/segmentation/nuscenes/unet_nuscenes_adversarial.py",
        "weights": "../../scripts/segmentation/models/nuscenes/unet_nuscenes_adversarial",
    },
    "UGV_adv": {
        "model": "../../config/segmentation/ugv/unet_ugv_adversarial.py",
        "weights": "../../scripts/segmentation/models/ugv/unet_ugv_adversarial",
    },
    "CARLA_adv_mc": {
        "model": "../../config/segmentation/carla/unet_carla_adversarial_mc.py",
        "weights": "../../scripts/segmentation/models/carla/unet_carla_adversarial_mc",
    },
    "nuScenes_adv_mc": {
        "model": "../../config/segmentation/nuscenes/unet_nuscenes_adversarial_mc.py",
        "weights": "../../scripts/segmentation/models/nuscenes/unet_nuscenes_adversarial_mc",
    },
    "UGV_adv_mc": {
        "model": "../../config/segmentation/ugv/unet_ugv_adversarial_mc.py",
        "weights": "../../scripts/segmentation/models/ugv/unet_ugv_adversarial_mc",
    },
}


# set up datasets
dataset_cfgs = {
    "CARLA": "../../config/segmentation/__base__/datasets/carla.py",
    "nuScenes": "../../config/segmentation/__base__/datasets/nuscenes.py",
    "UGV": "../../config/segmentation/__base__/datasets/ugv.py",
    "CARLA_adv": "../../config/segmentation/__base__/datasets/carla_adversarial.py",
    "nuScenes_adv": "../../config/segmentation/__base__/datasets/nuscenes_adversarial.py",
    "UGV_adv": "../../config/segmentation/__base__/datasets/ugv_adversarial.py",
}


In [ ]:
from avstack.config import Config
from fov.segmentation.utils import get_dataset, get_unet_model


# preload all the models
models = {
    m_name: get_unet_model(
        cfg=Config.fromfile(model_cfgs[m_name]["model"]),
        device=device,
        weight_dir=model_cfgs[m_name]["weights"],
    )
    for m_name in model_cfgs.keys()
}


# preload all the datasets (SM, dataset)
datasets = {
    d_name: get_dataset(
        cfg=Config.fromfile(dataset_cfgs[d_name]),
        device=device,
        split="val",
    )
    for d_name in dataset_cfgs.keys()
}

In [ ]:
import pickle
from collections import defaultdict
from eval_utils import test_model_on_dataset


# preset the dataset dictionary
all_results = defaultdict(dict)

# parameters for evaluation
n_frames_max = 200

# run cross-wise evaluations
for name_model_dataset in model_cfgs.keys():
    # load in the model
    model = models[name_model_dataset]

    # get the dataset to evaluate on
    for name_test_dataset in dataset_cfgs.keys():

        # load in the dataset
        SM, seg_dataset = datasets[name_test_dataset]

        # run the evaluation
        results = test_model_on_dataset(
            model=model,
            name_model_dataset=name_model_dataset,
            seg_dataset=seg_dataset,
            name_test_dataset=name_test_dataset,
            n_frames_max=n_frames_max,
        )

        # store results
        all_results[name_model_dataset][name_test_dataset] = results

# save results
with open("model_train_test.p", "wb") as f:
    pickle.dump(all_results, f)

## Make some results figures

In [ ]:
import os


# set up paths for saving
fig_dir = "figures/train_test"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
import cv2
import numpy as np


def fill_holes(image: np.ndarray):
    """Fills holes in a binary image."""
    # Threshold the image to ensure it's binary (if needed)
    # _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Copy the thresholded image
    im_floodfill = image.copy()

    # Mask used for flood fill.
    # Size needs to be 2 pixels larger than the image
    h, w = image.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Flood fill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255)

    # Invert flood filled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the filled holes
    im_out = image | im_floodfill_inv

    return im_out


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from avapi.visualize.snapshot import show_lidar_bev_with_boxes

from fov.segmentation.utils import get_polygon_model
from eval_utils import get_metrics

# set colormaps
plt.set_cmap("Greys")
cmap_base = colors.LinearSegmentedColormap.from_list("wt", ["white", "teal"])
cmap_overlay = colors.LinearSegmentedColormap.from_list(
    "wtb", ["white", "teal", "darkslategrey"]
)

# set up the frames to evaluate
idx_frame_eval = {"CARLA": 10, "nuScenes": 5, "UGV": 3, "CARLA_adv": 10, "nuScenes_adv": 6}

# loop over model/dataset pairs
for name_model_dataset in model_cfgs.keys():
    # load in the model
    model = models[name_model_dataset]

    # get the dataset to evaluate on
    for name_test_dataset in dataset_cfgs.keys():
        results = defaultdict(list)  # set the results for this combo

        # load in the dataset
        SM, seg_dataset = datasets[name_test_dataset]

        # run the evaluation
        print(
            f"\n----Evaluating model from {name_model_dataset.upper()} "
            f"on {name_test_dataset.upper()} data"
        )

        # get data
        frame = idx_frame_eval[name_test_dataset]
        pc_img, gt_mask = seg_dataset[frame]
        pc_img = torch.unsqueeze(pc_img, 0)
        pc_np = seg_dataset.get_pointcloud(frame)
        pc_avstack = seg_dataset.get_pointcloud_avstack(frame, SM)
        gt_poly_avstack = seg_dataset.get_polygon_avstack(frame, SM)
        metadata = seg_dataset.get_metadata(frame)

        # inference
        img_pred = model(pc_img, pc_np, metadata)

        # metrics
        this_results = get_metrics(img_pred, gt_mask, metadata)

        ###############################
        # visualizations
        ###############################
        
        # get polygon model for postproc
        poly_model = get_polygon_model(model="fast_ray_trace", extent=seg_dataset._extent, device="cpu")

        # set up directories
        fig_dir_this = os.path.join(
            fig_dir, f"model_{name_model_dataset}_dataset_{name_test_dataset}"
        )
        os.makedirs(fig_dir_this, exist_ok=True)
        fig_file_base = os.path.join(
            fig_dir_this,
            f"model_{name_model_dataset}_dataset_{name_test_dataset}_frame_{frame}_{{}}",
        )

        # avstack pc image
        pc_img_avstack = show_lidar_bev_with_boxes(
            pc=pc_avstack,
            boxes=[],
            return_image=True,
            show=True,
            scale_return_image=True,
        )
        cv2.imwrite(
            fig_file_base.format("pc_img_in_avstack.png"),
            cv2.cvtColor(pc_img_avstack, cv2.COLOR_BGR2RGB),
        )

        # avstack pc image with fov (convert fov image to polygon)
        pc_img_avstack_fov = show_lidar_bev_with_boxes(
            pc=pc_avstack,
            boxes=[],
            background_color="black",
            fov=gt_poly_avstack,
            fov_filled=True,
            fov_filled_alpha=0.5,
            return_image=True,
            show=True,
            scale_return_image=True,
        )
        cv2.imwrite(
            fig_file_base.format("pc_img_in_with_fov_avstack.png"),
            cv2.cvtColor(pc_img_avstack_fov, cv2.COLOR_BGR2RGB),
        )

        # pc image
        pc_img_np = pc_img.detach().cpu().numpy()[0, 0, ...]
        plt.imshow(pc_img_np, cmap=cmap_base)
        plt.axis("off")
        plt.savefig(fig_file_base.format("pc_img_in.png"))
        plt.savefig(fig_file_base.format("pc_img_in.pdf"))
        plt.show()

        # binarized pc image
        plt.imshow(pc_img_np > 0, cmap=cmap_overlay)
        plt.axis("off")
        plt.savefig(fig_file_base.format("mask_img_in.png"))
        plt.savefig(fig_file_base.format("mask_img_in.pdf"))
        plt.show()

        # ground truth mask
        gt_mask_np = gt_mask.detach().cpu().numpy()[0, ...]
        plt.imshow(gt_mask_np, cmap=cmap_base)
        plt.axis("off")
        plt.savefig(fig_file_base.format("gt_mask.png"))
        plt.savefig(fig_file_base.format("gt_mask.pdf"))
        plt.show()

        # predicted image
        img_pred_np = img_pred.detach().cpu().numpy()[0, 0, ...]
        plt.imshow(img_pred_np, cmap=cmap_base)
        plt.axis("off")
        plt.savefig(fig_file_base.format("img_pred.png"))
        plt.savefig(fig_file_base.format("img_pred.pdf"))
        plt.show()

        # predicted image binarized
        threshold = 0.7
        plt.imshow(img_pred_np > threshold, cmap=cmap_base)
        plt.axis("off")
        plt.savefig(fig_file_base.format("mask_pred.png"))
        plt.savefig(fig_file_base.format("mask_pred.pdf"))
        plt.show()

        # predicted image closed binarized
        closed_image_bin = fill_holes((img_pred_np > threshold).astype(np.uint8))
        plt.imshow(closed_image_bin > threshold, cmap=cmap_base)
        plt.axis("off")
        plt.savefig(fig_file_base.format("mask_pred_closed.png"))
        plt.savefig(fig_file_base.format("mask_pred_closed.pdf"))
        plt.show()

        # overlay points on closed binarized image
        img_pred_overlay = closed_image_bin.astype(bool) + 2 * (pc_img_np > 0)
        plt.imshow(np.minimum(2, img_pred_overlay), cmap=cmap_overlay)
        plt.axis("off")
        plt.savefig(fig_file_base.format("mask_pred_overlay.png"))
        plt.savefig(fig_file_base.format("mask_pred_overlay.pdf"))
        plt.show()

        # fit a polygon over it to use for AVstack image
        pts_avstack = seg_dataset.img_to_pts_avstack(img_pred_np, metadata, SM, threshold=0.7)
        poly_pred = poly_model.model(pts_avstack)
        pc_img_pred_avstack_fov = show_lidar_bev_with_boxes(
            pc=pc_avstack,
            boxes=[],
            background_color="black",
            fov=poly_pred,
            fov_filled=True,
            fov_filled_alpha=0.5,
            return_image=True,
            show=True,
            scale_return_image=True,
        )
        cv2.imwrite(
            fig_file_base.format("mask_pred_overlay_avstack.png"),
            cv2.cvtColor(pc_img_avstack_fov, cv2.COLOR_BGR2RGB),
        )